Calculates discounted cashflows for Fixed-Floating and OIS (TRAINING DATA)

In [29]:
# 1. Imports
import os
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# 2. Configuration
your_path = r'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'
train_folder_path = your_path + '6.Active Data/Train Model Data/'
test_folder_path = your_path + '6.Active Data/Test Data/'

# 3. Functions
def convert_to_usd(row):
    return row['leg1NotionalAmount'] * exchange_rates.get(row['leg1NotionalCurrency'], 1)

def calculate_cashflows(row, period, freq):
    cashflows = []
    start_date = row['effectiveDate']
    end_date = row['expirationDate']
    
    if row[period] == 'MNTH':
        delta = relativedelta(months=row[freq])
    elif row[period] == 'YEAR':
        delta = relativedelta(years=row[freq])
    elif row[period] == 'DAIL':
        delta = timedelta(days=row[freq])
    else:
        raise ValueError(f"Unknown frequency period: {row[period]}")
    
    while start_date < end_date:
        cashflows.append(start_date)
        start_date += delta

    return cashflows

def convert_to_months(value):
    if "month" in value:
        return int(value.split()[0])
    elif "year" in value:
        return int(value.split()[0]) * 12
    else:
        return 0

def discount_rate_curve(date, bump):
    months_to_maturity = (date - datetime.now()).days // 30
    if months_to_maturity < 0:
        rate = 3.5 # 3.5% Interest since the day the payment was made until todays date.
    else:
        reindexed_df = float_rates_df.set_index('MonthsToMaturity').reindex(float_rates_df['MonthsToMaturity'], method='nearest')
        rate = reindexed_df['Rate'].get(months_to_maturity, reindexed_df['Rate'].iloc[0]) + bump
    return rate / 100

def get_floating_rate(months_to_maturity):
    nearest_index = float_rates_df['MonthsToMaturity'].sub(months_to_maturity).abs().idxmin()
    return float_rates_df.loc[nearest_index, 'Rate']

def calculate_discounted_cashflow(row, type, bump):
    total_discounted_cashflow = 0
    months_to_maturity = (row['expirationDate'] - row['effectiveDate']).days // 30
    for date in row['cashflow_dates']:
        if type == 'float':
            float_rate = get_floating_rate(months_to_maturity) / 100
            cashflow = row['leg1NotionalAmount'] * float_rate
        else:
            cashflow = row['leg1NotionalAmount'] * row['leg1FixedRate']
        time_difference = (date - datetime.now()).days / 365.0
        discounted_cashflow = cashflow / (1 + discount_rate_curve(date, bump))**time_difference
        total_discounted_cashflow += discounted_cashflow
    return total_discounted_cashflow

def main (folder_path, your_path):
    # 4. Data Loading
    # Load trade data
    files = [f for f in os.listdir(folder_path) if f != '.DS_Store']
    data_file = files[0][:-4]
    data = pd.read_csv(your_path + "2.Cleaned/" + data_file +'_Cleaned.csv')

    # Load exchange rates
    exchange_rates_df = pd.read_csv(your_path + 'exchange_rates.csv')
    exchange_rates = dict(zip(exchange_rates_df['Currency'], exchange_rates_df['Rate_to_USD']))

    # Load floating rates
    euribor_df = pd.read_csv(your_path + 'EURIBOR.csv')

    # 5. Data Transformation and Filtering
    filtered_data = data[
        (data['leg1NotionalCurrency'].isin(['EUR', 'USD', 'GBP'])) & 
        (data['leg1UnderlyingAssetOrContractType'].isin(['Fixed-Floating', 'OIS']))
    ]
    df = pd.DataFrame(filtered_data)
    df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
    df['expirationDate'] = pd.to_datetime(df['expirationDate'])

    euribor_df['MonthsToMaturity'] = euribor_df['timeToMaturity'].apply(convert_to_months)
    float_rates_df = euribor_df[['MonthsToMaturity', 'Rate']].copy()

    df['cashflow_dates'] = df.apply(
        lambda row: 
            calculate_cashflows(row, 'leg1FixedRatePaymentFrequencyPeriod', 'leg1FixedRatePaymentFrequencyMultiplier') 
            if row['leg1UnderlyingAssetOrContractType'] == 'Fixed-Floating' 
            else (calculate_cashflows(row, 'leg2UnderlierTenorPeriod', 'leg2UnderlierTenorMultiplier') 
                if row['leg1UnderlyingAssetOrContractType'] == 'OIS' 
                else None), 
        axis=1)

    # 6. Main Operations
    df['leg1NotionalAmountUSD'] = df.apply(convert_to_usd, axis=1)
    df['MtM_leg1'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'fixed', 0), axis=1)
    df['MtM_leg2'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float', 0), axis=1)
    df['MtM_leg2_bumped'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float', 0.01), axis=1)
    df['total_delta'] = (df['MtM_leg2'] - df['MtM_leg2_bumped']).abs()
    print(df['cashflow_dates'])
    # 7. Export/Output
    df.drop(columns=['cashflow_dates', 'MtM_leg2_bumped'], inplace=True)
    df.to_csv(your_path + '3.Cash_Risk/' + data_file + '_Cash_Risk.csv', index=False)

main(train_folder_path, your_path)
#main(test_folder_path, your_path)


/var/folders/rq/30rpw85n53d2zxnt_c5qnrx40000gn/T/ipykernel_80351/2207756342.py:91: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
/var/folders/rq/30rpw85n53d2zxnt_c5qnrx40000gn/T/ipykernel_80351/2207756342.py:92: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['expirationDate'] = pd.to_datetime(df['expirationDate'])


0    [2022-12-21 00:00:00, 2023-06-21 00:00:00, 202...
1    [2022-12-21 00:00:00, 2023-06-21 00:00:00, 202...
2    [2022-12-21 00:00:00, 2023-06-21 00:00:00, 202...
3    [2022-12-21 00:00:00, 2023-06-21 00:00:00, 202...
4    [2022-12-21 00:00:00, 2023-06-21 00:00:00, 202...
5    [2022-12-21 00:00:00, 2023-06-21 00:00:00, 202...
Name: cashflow_dates, dtype: object


Now the same for the Test data